In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

import torch
import torchvision
import torch.optim as optim
import torch.nn as nn
from torchvision import datasets, transforms
print("PyTorch Version: ",torch.__version__)
print("Torchvision Version: ",torchvision.__version__)


!mkdir /kaggle/working/Tomato
!mkdir /kaggle/working/Tomato/test
!mkdir /kaggle/working/Tomato/train

!cp -r /kaggle/input/plant-diseases/dataset_itr2/dataset_itr2/test/Tomato* /kaggle/working/Tomato/test
!cp -r /kaggle/input/plant-diseases/dataset_itr2/dataset_itr2/train/Tomato* /kaggle/working/Tomato/train

PyTorch Version:  1.11.0
Torchvision Version:  0.12.0


In [2]:
train_path='/kaggle/working/Tomato/train'
test_path='/kaggle/working/Tomato/test'
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [3]:
transfrom = transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])

In [4]:
from torchvision.datasets import ImageFolder

test_dataset=ImageFolder(test_path,transform=transfrom)
train_dataset=ImageFolder(train_path,transform=transfrom)

batch = 32
n_iter = 3000
epochs = n_iter / (len(train_dataset) / batch)
epochs = int(epochs)

trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch, shuffle=True, num_workers=2)

testloader = torch.utils.data.DataLoader(test_dataset, batch_size=batch, shuffle=False, num_workers=2)

In [12]:
AlexNet_model = torch.hub.load('pytorch/vision:v0.6.0', 'alexnet', pretrained=True)


AlexNet_model.eval()
model = AlexNet_model.cuda()

Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.6.0


In [13]:
from torch.autograd import Variable
num_epochs = 10
criterion = nn.CrossEntropyLoss()
learning_rate = 0.01
optimizer = optim.Adam(AlexNet_model.parameters(), lr=0.001)

iter = 0
for epoch in range(int(num_epochs)):
    for i, (images, labels) in enumerate(trainloader):
        images = Variable(images.cuda())
        labels = Variable(labels.cuda())
        
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        iter += 1
        
        if iter % 500 == 0:
            correct = 0
            total = 0
            for images, labels in testloader:
                images = Variable(images.cuda())
                outputs = model(images)
                
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                
                if torch.cuda.is_available():
                    correct += (predicted.cpu() == labels.cpu()).sum()
                else:
                    correct += (predicted == labels).sum()
                
            accuracy = 100 * correct / total
            print('Iteration: {}. Loss {}. Accuracy: {}'.format(iter, loss.data, accuracy))

Iteration: 500. Loss 1.3370039463043213. Accuracy: 56.353363037109375
Iteration: 1000. Loss 0.60061115026474. Accuracy: 75.50992584228516
Iteration: 1500. Loss 0.41946089267730713. Accuracy: 75.7786636352539
Iteration: 2000. Loss 0.6420803666114807. Accuracy: 81.73925018310547
Iteration: 2500. Loss 0.42754411697387695. Accuracy: 83.42061614990234
Iteration: 3000. Loss 0.8242958784103394. Accuracy: 78.8726577758789
Iteration: 3500. Loss 0.5978414416313171. Accuracy: 83.38616180419922
Iteration: 4000. Loss 0.4555121064186096. Accuracy: 87.72050476074219
Iteration: 4500. Loss 0.27992281317710876. Accuracy: 86.04602813720703
Iteration: 5000. Loss 0.24130098521709442. Accuracy: 88.7403564453125
Iteration: 5500. Loss 0.17565330862998962. Accuracy: 88.74724578857422
Iteration: 6000. Loss 0.6015636920928955. Accuracy: 88.25110626220703
Iteration: 6500. Loss 0.34836477041244507. Accuracy: 88.42337036132812
Iteration: 7000. Loss 0.6701751947402954. Accuracy: 83.48263549804688
Iteration: 7500. Lo

In [ ]:
from sklearn.metrics import confusion_matrix


In [ ]:
import torch
from torch.autograd import Variable
import numpy as np
import pylab as pl
import torch.nn.init as init

In [ ]:
import torch.optim as optim
import torch.nn as nn

#Loss
criterion = nn.CrossEntropyLoss()

#Optimizer(SGD)
optimizer = optim.SGD(model.parameters(), lr=0.001)

for epoch in range(epochs):  # loop over the dataset multiple times
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data[0].to(device), data[1].to(device)
        inputs = Variable(inputs.cuda())
        labels = Variable(labels.cuda())

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        output = model(inputs)
        loss = criterion(output, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training of AlexNet')